In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np


In [2]:
def get_name(soup):
    try:
        title=soup.find('span',attrs={'id':"productTitle"})
        title_value = title.text
        title_string = title_value.strip()
    except AttributeError:
        title_string = ""

    return title_string
def get_price(soup):
    try:
        price=soup.find('span',attrs={'class':"a-price-whole"}).text
    except AttributeError:
        price= ""

    return price
def get_rating(soup):
    try:
        rating=soup.find('span',attrs={'class':"a-icon-alt"}).text
    except AttributeError:
        rating=''
    return rating
def get_reviews(soup):
    try:
        reviews=soup.find('span',attrs={'id':"acrCustomerReviewText"}).text
    except AttributeError:
        reviews=''
    return reviews

    

In [3]:
url="https://www.amazon.in/s?k=bags&crid=2M096C61O4MLT&qid=1653308124&sprefix=ba%2Caps%2C283&ref=sr_pg_1"
HEADERS=({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'})
client=requests.session()
d={'product url':[],'product name':[],'product price':[],'rating':[],'no of reviews':[]}
i=1
while True:
    webpage=client.get(url,headers=HEADERS)
    soup=BeautifulSoup(webpage.content,"html.parser")
    links=soup.find_all("a",attrs={'class':"a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"})
    links_list=[]
    for link in links:
        links_list.append(link.get('href'))
    
    for link in links_list:
        new_webpage=requests.get("http://www.amazon.in"+link,headers=HEADERS)
        new_soup=BeautifulSoup(new_webpage.content,"html.parser")
        apin=new_soup.find_all('span',attrs={'class':"a-list-item"}).text
        d['product url'].append("http://www.amazon.in"+link)
        d['product name'].append(get_name(new_soup))
        d['product price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['no of reviews'].append(get_reviews(new_soup))      

    nextlink=soup.find('a',attrs={'class':"s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"})
    try:
        url="http://www.amazon.in"+nextlink.get('href')
    except AttributeError:
        pass
    #count=int(soup.find('span',attrs={'class':"s-pagination-item s-pagination-disabled"}).text)
    i+=1

    if(i>=2):
        break
amazon_df = pd.DataFrame.from_dict(d)
amazon_df['product name'].replace('', np.nan, inplace=True)
amazon_df = amazon_df.dropna(subset=['product name'])
amazon_df.to_csv("amazon_data.csv", header=True, index=False)



    

In [ ]:
webpage